xgboost total points

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv("cleaned_merged_seasons.csv")

<ipython-input-2-ce1e41c9b2f4>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_merged_seasons.csv")


In [3]:
print(data.head())

  season_x                name position team_x  assists  bonus  bps  \
0  2016-17     Aaron Cresswell      DEF    NaN        0      0    0   
1  2016-17        Aaron Lennon      MID    NaN        0      0    6   
2  2016-17        Aaron Ramsey      MID    NaN        0      0    5   
3  2016-17  Abdoulaye Doucouré      MID    NaN        0      0    0   
4  2016-17        Adam Forshaw      MID    NaN        0      0    3   

   clean_sheets  creativity  element  ...  team_h_score  threat  total_points  \
0             0         0.0      454  ...           2.0     0.0             0   
1             0         0.3      142  ...           1.0     0.0             1   
2             0         4.9       16  ...           3.0    23.0             2   
3             0         0.0      482  ...           1.0     0.0             0   
4             0         1.3      286  ...           1.0     0.0             1   

   transfers_balance  transfers_in transfers_out  value  was_home  \
0                

In [4]:
# valeurs manquantes
data = data.dropna(subset=['team_x'])

# Normalise
features_to_normalize = ["creativity", "ict_index", "influence", "threat"]
data[features_to_normalize] = data[features_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

# Encode
data_encoded = pd.get_dummies(data, columns=["season_x", "position", "team_x", "kickoff_time", "opp_team_name"], drop_first=True)

# data type int
data_encoded['was_home'] = data_encoded['was_home'].astype(int)

# Retirer la colonne "name"
player_names = data_encoded['name'].reset_index(drop=True)
data_encoded = data_encoded.drop('name', axis=1)

<ipython-input-4-d8dac73b6c8b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[features_to_normalize] = data[features_to_normalize].apply(lambda x: (x - x.min()) / (x.max() - x.min()))


In [5]:
print(player_names)

0                  Aaron Connolly
1                 Aaron Cresswell
2                      Aaron Mooy
3                  Aaron Ramsdale
4              Abdoulaye Doucouré
                   ...           
76312                Oliver Skipp
76313              Ryan Sessegnon
76314                Ashley Young
76315    Jeremy Sarmiento Morante
76316              Philip Billing
Name: name, Length: 76317, dtype: object


In [6]:
# features (X) et cible (y)
X = data_encoded.copy()

# Ajouter les colonnes 'team_x' et 'position' à X
columns_to_add = ['team_x', 'position']
if all(col in data_encoded.columns for col in columns_to_add):
    X[columns_to_add] = data_encoded[columns_to_add]

# Supprimer la colonne 'total_points' de X
X = X.drop('total_points', axis=1)

# Cible (y)
y = data_encoded['total_points']

In [7]:
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(X, y, player_names, test_size=0.2, random_state=42)


In [13]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Définir les paramètres du modèle XGBoost
params = {
    'objective': 'reg:squarederror',
    'max_depth': 3,
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric': 'rmse'
}

# Créer un modèle XGBoost compatible avec scikit-learn
xgb_model = XGBRegressor(**params)

# Utiliser cross_val_score pour effectuer une validation croisée
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Convertir les scores en MSE positif (puisque scikit-learn renvoie le négatif de MSE)
cv_scores = -cv_scores

# Afficher les résultats de la validation croisée
print("Cross-validated MSE scores:", cv_scores)
print("Mean MSE:", np.mean(cv_scores))


Cross-validated MSE scores: [0.10744126 0.12599157 0.11981205 0.10873473 0.11639207]
Mean MSE: 0.11567433737002156


In [14]:
# Convertir les données en format XGBoost DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Définir les paramètres du modèle XGBoost
params = {
    'objective': 'reg:squarederror',  # Regression task
    'max_depth': 3,                    # Profondeur maximale de l'arbre
    'learning_rate': 0.1,              # Taux d'apprentissage
    'n_estimators': 100,               # Nombre d'arbres à construire
    'subsample': 0.8,                  # Sous-échantillonnage des lignes
    'colsample_bytree': 0.8,           # Sous-échantillonnage des colonnes
    'eval_metric': 'rmse'              # Métrique d'évaluation (Root Mean Squared Error)
}

# Entraîner le modèle XGBoost
model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dtest, 'test')], early_stopping_rounds=50, verbose_eval=False)

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(dtest)

# Évaluer les performances du modèle
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:06:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Mean Squared Error: 0.012750229131956182
Mean Absolute Error: 0.039426968431273086
R-squared: 0.9978401455923648


In [15]:
player_names_test = player_names_test.reset_index(drop=True)

results_df = pd.DataFrame({'Name': player_names_test, 'Predicted_Total_Points': y_pred})

print(results_df)

                    Name  Predicted_Total_Points
0           Daniel James                1.011807
1           Nicolas Pépé                2.046713
2             Sam Waller                0.000013
3             Bernd Leno                0.000662
4       Ezri Konsa Ngoyo                6.020627
...                  ...                     ...
15259  Aaron Wan-Bissaka               -0.000852
15260       Divock Origi               -0.000398
15261   Joseph Whitworth                0.000013
15262       Alex Mighten                0.002789
15263  Christian Pulisic                4.917779

[15264 rows x 2 columns]
